## Overview of Gaussian Naive Bayes

Naive Bayes is a family of probabilistic classification methods built on generative modeling, where the goal is to characterize how the features and labels are jointly distributed. Instead of directly learning the conditional probability $P(y \mid X)$, the method models the joint structure by estimating the class prior distribution together with the class-conditional distributions of the features. The posterior probability used for classification is then obtained via Bayes' rule.

At the core of Naive Bayes is the naive conditional independence assumption: given the class label, all features are treated as independent. While this assumption is often violated in real-world data, it greatly simplifies the modeling process, allowing all parameters to be estimated efficiently using simple empirical statistics computed within each class.

Different variants of Naive Bayes arise from different assumptions about the class-conditional feature distributions. For example, Multinomial and Bernoulli Naive Bayes are commonly used for text data, while Categorical Naive Bayes applies to discrete, non-numeric features. 

In this project, we focus on Gaussian Naive Bayes, which assumes that each continuous feature follows a class-conditional normal distribution. This makes GaussianNB particularly suitable for datasets with real-valued features and provides a simple yet effective generative model for continuous data.

| Variant | Feature Type | Distribution Assumption | Typical Use Case |
|--------|--------------|-------------------------|------------------|
| **Gaussian Naive Bayes** | Continuous numerical features | Gaussian (normal) distribution | Sensor data, measurements, general continuous features |
| **Multinomial Naive Bayes** | Count-based features | Multinomial distribution | Text classification, word counts (BoW, TF) |
| **Bernoulli Naive Bayes** | Binary (0/1) features | Bernoulli distribution | Word presence/absence, binary indicators |
| **Categorical Naive Bayes** | General categorical features | Categorical distribution | Discrete categorical variables (color, city, etc.) |


## Advantages and Disadvantages of Gaussian Naive Bayes

### **Advantages**

**1. Extremely fast training**  
GaussianNB uses closed-form estimates for class means and variances, requiring no iterative optimization. 

**2. Performs well on small datasets**  
Estimating Gaussian parameters requires relatively few samples, allowing GNB to work effectively even when data is limited.

**3. Simple and interpretable**  
The contribution of each feature to the final prediction can be clearly understood through class-wise means, variances, and log-likelihoods.

**4. Efficient in high-dimensional spaces**  
Due to the independence assumption, the computational cost scales linearly with the number of features, making GNB suitable for high-dimensional data.


### **Disadvantages**

**1. Independence assumption rarely holds**  
The naive assumption that features are conditionally independent is often violated in real-world data, which can degrade model performance.

**2. Sensitive to distribution mismatch**  
GNB assumes each feature follows a Gaussian distribution within each class. Strongly skewed, multimodal, or heavy-tailed distributions may lead to poor results.

**3. Limited model flexibility**  
The decision boundaries are quadratic, restricting the model’s ability to capture more complex nonlinear structures.

**4. Assumes each feature dimension is an independent univariate Gaussian**  
GNB does not capture correlations between features (i.e., it does not model full multivariate Gaussian distributions), which can be limiting when feature interactions are important.

## Problem Assumptions

Gaussian Naive Bayes is a **generative classifier**:  
it models the joint distribution \(p(x, y)\) and then uses Bayes’ rule
to predict the most likely class \(y\) for a new feature vector \(x\).

---

### Notation

- \(x \in \mathbb{R}^d\): feature vector with \(d\) continuous attributes  
- \(y \in \{c_1, \dots, c_K\}\): class label (categorical)  
- \(\mu_{c,j}, \sigma_{c,j}^2\): mean and variance of feature \(j\) in class \(c\)

---

### Assumption 1 — Class-conditional Gaussian distributions

For each class \(c\) and each feature \(j\), the feature value follows
a **Gaussian distribution** within that class:

$$
x_j \mid (y = c) \sim \mathcal{N}(\mu_{c,j}, \sigma_{c,j}^2).
$$

So, conditioned on the class, each feature is modeled
as a one-dimensional normal variable with its own mean and variance.

---

### Assumption 2 — Naive conditional independence & generative model

Given the class label \(y = c\), all features are assumed to be
**conditionally independent**:

$$
p(x \mid y = c)
= \prod_{j=1}^d p(x_j \mid y = c).
$$

Using this together with the class prior \(p(y = c)\),
the **generative joint model** factorizes as

$$
p(x, y = c)
= p(y = c)\, p(x \mid y = c)
= \pi_c \prod_{j=1}^d \mathcal{N}(x_j \mid \mu_{c,j}, \sigma_{c,j}^2),
$$

where \(\pi_c = p(y = c)\) is the class prior.

## Representation

Gaussian Naive Bayes is a **generative classifier**.  
It models how the pair $(x, y)$ is generated, and then uses Bayes’ rule to
predict the most likely class for a new feature vector $x$.

---

### Bayes rule

We start from Bayes’ rule for a single class $c$:

$$
P(y = c \mid x)
= \frac{P(x \mid y = c)\, P(y = c)}{P(x)} .
$$

Here

- $P(y = c \mid x)$: posterior — probability that the label is class $c$ given $x$  
- $P(x \mid y = c)$: likelihood of seeing features $x$ under class $c$  
- $P(y = c)$: class prior  
- $P(x)$: evidence (normalization term, same for all classes when we compare them)

---

### Class prior $P(y = c)$

We estimate the prior by counting how often each class appears in the training set.
Let $N_c$ be the number of samples with label $c$, and $N$ the total number of
samples:

$$
P(y = c) \approx \hat P(y = c)
= \frac{N_c}{N} .
$$

So the prior just measures how common class $c$ is in the data.

---

### Likelihood of features $P(x \mid y = c)$

Let the feature vector be $x = (x_1, \dots, x_d)$.

1. **Naive conditional independence**

   Given the class $y = c$, features are assumed conditionally independent:

   $$
   P(x \mid y = c)
   = \prod_{j=1}^d P(x_j \mid y = c) .
   $$

2. **Class-conditional Gaussian distributions**

   For each class $c$ and feature $j$, we assume a univariate Gaussian:

   $$
   x_j \mid y = c \sim \mathcal{N}(\mu_{c,j}, \sigma_{c,j}^2),
   $$

   so the per-feature likelihood is

   $$
   P(x_j \mid y = c)
   = \mathcal{N}(x_j \mid \mu_{c,j}, \sigma_{c,j}^2)
   = \frac{1}{\sqrt{2\pi\sigma_{c,j}^2}}
     \exp\!\left(
       -\frac{(x_j - \mu_{c,j})^2}{2\sigma_{c,j}^2}
     \right).
   $$

   Combining these, the total likelihood is

   $$
   P(x \mid y = c)
   = \prod_{j=1}^d
     \mathcal{N}(x_j \mid \mu_{c,j}, \sigma_{c,j}^2).
   $$

---

### Final representation (prediction rule)

For prediction we choose the class with the largest posterior probability:

$$
\hat{y}
= \arg\max_{c} P(y = c \mid x).
$$

Using Bayes’ rule and dropping $P(x)$, which is the same for all classes,
this becomes

$$
\hat{y}
= \arg\max_{c} P(x \mid y = c)\, P(y = c)
= \arg\max_{c}
\left[
  \prod_{j=1}^d
  \mathcal{N}(x_j \mid \mu_{c,j}, \sigma_{c,j}^2)
\right]
\frac{N_c}{N}.
$$

In code, we usually work in **log space**, replacing products by sums:

$$
\log P(y = c \mid x)
= \log P(y = c)
+ \sum_{j=1}^d \log \mathcal{N}(x_j \mid \mu_{c,j}, \sigma_{c,j}^2)
\quad \text{(up to an additive constant)} ,
$$

and still predict

$$
\hat{y} = \arg\max_{c} \log P(y = c \mid x).
$$

## Loss

We train Gaussian Naive Bayes by **maximum likelihood**,  
which is equivalent to **minimizing the negative log-likelihood (NLL)**.

---

### General form

Given a parametric model with parameters $\Theta$ and training data
$\{(x_i, y_i)\}_{i=1}^n$, the likelihood of the data is

$$
p_\Theta(\{x_i, y_i\}_{i=1}^n)
= \prod_{i=1}^n p_\Theta(y_i, x_i).
$$

The **negative log-likelihood loss** is

$$
\mathcal{L}_{\text{NLL}}(\Theta)
= - \sum_{i=1}^n \log p_\Theta(y_i, x_i).
$$

Minimizing $\mathcal{L}_{\text{NLL}}$ is the same as maximizing the likelihood.

---

### NLL for Gaussian Naive Bayes

For Gaussian NB, the joint model for a single sample is

$$
p_\Theta(x_i, y_i)
= \pi_{y_i}
  \prod_{j=1}^d
  \mathcal{N}(x_{i,j} \mid \mu_{y_i,j}, \sigma_{y_i,j}^2),
$$

where

- $\pi_c = P(y = c)$ is the class prior,  
- $\mu_{c,j}$ and $\sigma_{c,j}^2$ are the mean and variance of feature $j$ in class $c$.

Plugging this into the general NLL and expanding the Gaussian log-density gives

$$
\mathcal{L}_{\text{NLL}}(\Theta)
=
\sum_{i=1}^n
\left[
-\log \pi_{y_i}
+
\sum_{j=1}^d
\left(
\frac{1}{2}\log\!\big(2\pi \sigma_{y_i,j}^2\big)
+
\frac{(x_{i,j} - \mu_{y_i,j})^2}{2\sigma_{y_i,j}^2}
\right)
\right].
$$

- The term $-\log \pi_{y_i}$ comes from the **class prior**.  
- The inner sum over $j$ comes from the **Gaussian likelihood** of each feature.

Because Gaussian NB has **closed-form MLE solutions** for $\pi_c$, $\mu_{c,j}$, and
$\sigma_{c,j}^2$, we usually do **not** run gradient descent on this loss in practice;
instead we compute the empirical counts, means, and variances that minimize it.

## Optimizer(Not Real)

Unlike discriminative models such as logistic regression or neural networks,  
Gaussian Naive Bayes does not require an iterative optimizer.  
Although the model minimizes the negative log-likelihood (NLL), the optimal parameters
have closed-form maximum likelihood solutions.

For each class $c$ and feature $j$, the MLE updates are:

$$
\hat{\pi}_c = \frac{N_c}{N},
$$
where $N_c$ is the number of samples in class $c$.

$$
\hat{\mu}_{c,j}
= \frac{1}{N_c} \sum_{i : y_i = c} x_{i,j}.
$$

$$
\hat{\sigma}_{c,j}^2
= \frac{1}{N_c} \sum_{i : y_i = c} (x_{i,j} - \hat{\mu}_{c,j})^2.
$$

These formulas directly minimize the NLL loss, so training requires only computing
class counts, sample means, and sample variances—no gradient descent, no iterative
optimization, and no numerical solver.


## Pseudo-code

Training Gaussian Naive Bayes model  
**Require:** Training dataset $(X, y)$, smoothing parameter $\text{var\_smoothing}$  
**Ensure:** Class priors $P(c)$, means $\mu_{c,j}$, variances $\sigma_{c,j}^2$  

1:  $N \leftarrow$ number of samples in $X$  
2:  **for** $j = 1 \dots d$ **do**  
3:  $\hat{\sigma}_j^2 \leftarrow \frac{1}{N} \sum_{i=1}^N (X_{i,j} - \bar{X}_j)^2$  
4:  **end for**  
5:  $v_{\max} \leftarrow \max_{1 \le j \le d} \hat{\sigma}_j^2$  
6:  $\epsilon \leftarrow \text{var\_smoothing} \cdot v_{\max}$  

7:  $C \leftarrow \text{unique}(y)$  
8:  **for each** class $c \in C$ **do**  
9:   $S_c \leftarrow \{\, i : y_i = c \,\}$  
10:   $N_c \leftarrow |S_c|$  
11:   $P(c) \leftarrow \dfrac{N_c}{N}$  
12:   **for** $j = 1 \dots d$ **do**  
13:    $X_{c,j} \leftarrow \{\, X_{i,j} : i \in S_c \,\}$  
14:    $\mu_{c,j} \leftarrow \dfrac{1}{N_c} \sum_{i \in S_c} X_{i,j}$  
15:    $\sigma_{c,j}^2 \leftarrow \dfrac{1}{N_c} \sum_{i \in S_c} (X_{i,j} - \mu_{c,j})^2 + \epsilon$  
16:   **end for**  
17: **end for**  

---

Predicting with Gaussian Naive Bayes  
**Require:** Model parameters $P(c)$, $\mu_{c,j}$, $\sigma_{c,j}^2$  
**Ensure:** Predicted label $\hat{y}$  

1:  **for each** class $c \in C$ **do**  
2:  $\text{score}(c) \leftarrow \log P(c)$  
3:  **for** $j = 1 \dots d$ **do**  
4:   $\ell \leftarrow -\frac{1}{2}\log(2\pi\sigma_{c,j}^2) 
      \;-\; \frac{(x_j - \mu_{c,j})^2}{2\sigma_{c,j}^2}$  
5:   $\text{score}(c) \leftarrow \text{score}(c) + \ell$  
6:  **end for**  
7: **end for**  
8:  $\hat{y} \leftarrow \arg\max_{c} \text{score}(c)$

In [58]:
import numpy as np
class GaussianNaiveBayes:
    """
    Gaussian Naive Bayes classifier.

    This classifier assumes all features follow a Gaussian distribution and
    estimates per-class means and variances. Variance smoothing is applied
    using the formula: epsilon = var_smoothing * max(feature_variances)

    All probability calculations are done in log space to avoid numerical
    underflow when probabilities become very small. During prediction the model
    converts log probabilities back into normal probabilities and normalizes
    them so that each row sums to one.

    Feature likelihoods
    ----------
    For Gaussian numeric features:
        log_pdf = -0.5 * ( log(2 * pi * variance) + ((x - mean)^2) / variance )

    The model uses maximum likelihood estimates of the mean and variance
    for each feature within each class, with an additional variance smoothing
    term added for numerical stability.

    Parameters
    ----------
    var_smoothing : float, optional (default=1e-9)
        Portion of the largest feature variance added to each variance estimate
        for numerical stability.

    Example
    -------
    >>> X = np.array([[1.0, 2.0],
    ...               [1.2, 1.9],
    ...               [3.2, 4.8],
    ...               [3.0, 5.1]])
    >>> y = np.array(['A', 'A', 'B', 'B'])
    >>> model = GaussianNaiveBayes()
    >>> model.fit(X, y)
    >>> model.predict([[1.1, 2.0]])
    array(['A'], dtype='<U1')
    """


    def __init__(self, var_smoothing=1e-9):
        """
        Initialize a Gaussian Naive Bayes classifier.

        This constructor sets up the model hyperparameters and internal
        data structures that will be populated during fitting.

        Parameters
        ----------
        var_smoothing : float, default=1e-9
            Non-negative smoothing factor added to feature variances
            for numerical stability. The actual smoothing term is
            scaled by the maximum variance across all features.

        Attributes
        ----------
        classes_ : ndarray of shape (n_classes,)
            Array of unique class labels observed during fitting.

        log_priors_ : dict
            Dictionary mapping each class label to its log prior
            probability log P(Y = c).

        gaussian_means_ : dict
            Nested dictionary storing per-class per-feature Gaussian means.
            gaussian_means_[c][j] = mean of feature j for class c.

        gaussian_vars_ : dict
            Nested dictionary storing per-class per-feature Gaussian variances
            (after variance smoothing).
            gaussian_vars_[c][j] = variance of feature j for class c.

        Notes
        -----
        - All attributes except var_smoothing are initialized empty and
        filled during the call to fit().
        """
        self.var_smoothing = var_smoothing
        self.classes_ = None
        self.log_priors_ = {}
        self.gaussian_means_ = {}
        self.gaussian_vars_ = {}

    # Likelihood helper functions
    def _gaussian_log_pdf(self, x, mean, var):
        """
        Compute the log probability density of a Gaussian distribution.

        This function evaluates the log of the Gaussian (normal) probability
        density function for a single feature value.

        The Gaussian log-PDF is given by:
            log N(x | mean, var)
            = -0.5 * [ log(2π · var) + (x - mean)^2 / var ]

        Parameters
        ----------
        x : float
            Observed feature value.

        mean : float
            Mean of the Gaussian distribution for the given class and feature.

        var : float
            Variance of the Gaussian distribution for the given class and feature.
            Assumed to be strictly positive after variance smoothing.

        Returns
        -------
        float
            Log probability density log P(x | mean, var).

        Notes
        -----
        - Computation is done entirely in log space for numerical stability.
        - This method is used as a building block for computing class-conditional
        log-likelihoods.
        """
        return -0.5 * (np.log(2.0 * np.pi * var) + ((x - mean) ** 2) / var)

    # Aggregator
    def _compute_log_likelihood(self, x_row, class_label):
        """
        Compute the total log-likelihood log p(x | c) of a single sample under
        a given class using the Gaussian Naive Bayes assumption.

        Under the Naive Bayes conditional independence (i.i.d.) assumption,
        the joint likelihood factorizes across features:
            p(x | c) = ∏_j p(x_j | c)
            log p(x | c) = ∑_j log p(x_j | c)

        Each feature likelihood p(x_j | c) is modeled as a univariate Gaussian
        distribution with class-specific mean and variance.

        Parameters
        ----------
        x_row : array-like of shape (n_features,)
            A single data point.
        class_label : object
            The class for which likelihood is being computed.

        Returns
        -------
        float
            The total log-likelihood log p(x | c), computed as the sum of 
            Gaussian log-densities across all features.
        
        Notes
        -----
            - This method operates fully in log space to ensure numerical stability.
            - The method assumes that Gaussian means and variances for each class
            have already been estimated and stored in:
                - self.gaussian_means_[class_label]
                - self.gaussian_vars_[class_label]

            Algorithm
            ---------
            1. Initialize log-likelihood accumulator log_l = 0
            2. For each feature j:
                a. Retrieve class-specific mean μ_{c,j} and variance σ²_{c,j}
                b. Compute log p(x_j | c) using a Gaussian log-pdf
                c. Add the result to log_l
            3. Return the accumulated log-likelihood

        Example
        -------
        >>> model = GaussianNaiveBayes()
        >>> model.gaussian_means_ = {'A': {0: 0.0}}
        >>> model.gaussian_vars_ = {'A': {0: 1.0}}
        >>> model._compute_log_likelihood([0.0], 'A')
        0.0

        >>> model.gaussian_means_ = {'A': {0: 0.0, 1: 1.0}}
        >>> model.gaussian_vars_  = {'A': {0: 1.0, 1: 1.0}}
        >>> model._compute_log_likelihood([0.0, 1.0], 'A')
        -1.8378770664093453
        """
        log_l = 0.0

        for j in range(len(x_row)):
            mean = self.gaussian_means_[class_label][j]
            var  = self.gaussian_vars_[class_label][j]
            log_l += self._gaussian_log_pdf(x_row[j], mean, var)

        return log_l

    # Fit
    def fit(self, X, y):
        """
        Fit the Gaussian Naive Bayes model using training data.

        This method estimates:
        - Class prior probabilities P(Y = c)
        - Per-class per-feature Gaussian parameters (mean and variance)

        under the Naive Bayes conditional independence assumption.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training feature matrix consisting of continuous numerical features.

        y : array-like of shape (n_samples,)
            Class labels corresponding to each training sample.

        Returns
        -------
        self : object
            Returns the fitted model instance.

        Raises
        ------
        ValueError
            If X contains missing values (NaN). This implementation assumes
            all features are fully observed.

        Notes
        -----
        - Variance smoothing is applied to each feature variance using:
            var_jc ← var_jc + var_smoothing * max_j Var(X_j)
        where Var(X_j) is the variance of feature j across the entire dataset.
        - All probability-related quantities are stored in log space.
        """
        X = np.asarray(X, float)
        y = np.asarray(y)

        if np.isnan(X).any():
            raise ValueError("GaussianNaiveBayes does not support missing values. "
                     "Please remove or impute missing data before fitting.")


        n_samples, n_features = X.shape
        self.classes_ = np.unique(y)

        feature_vars = X.var(axis=0)          # per-feature variance
        max_var = np.max(feature_vars)        # biggest variance across features
        epsilon = self.var_smoothing * max_var 

        # class priors
        for c in self.classes_:
            n_c = np.sum(y == c)
            self.log_priors_[c] = np.log(n_c / n_samples)

        # init Gaussian dicts
        for c in self.classes_:
            self.gaussian_means_[c] = {}
            self.gaussian_vars_[c] = {}

        # compute Gaussian parameters
        for c in self.classes_:
            Xc = X[y == c]

            means = Xc.mean(axis=0)
            vars_  = Xc.var(axis=0)

            # smoothing:
            # var += eps * global_variance
            smoothed_vars = vars_ + epsilon

            for j in range(n_features):
                self.gaussian_means_[c][j] = means[j]
                self.gaussian_vars_[c][j] = smoothed_vars[j]

        return self

    # Prediction
    def predict_log_proba(self, X):
        """
        Compute unnormalized log posterior scores log P(Y = c | X) for each sample
        and each class using Gaussian Naive Bayes.

        Using Bayes' rule, the posterior is proportional to the product of the
        class prior and the class-conditional likelihood:
            P(c | x) ∝ P(c) · P(x | c)

        Taking the logarithm gives:
            log P(c | x) = log P(c) + log P(x | c)

        where log P(x | c) is computed under the Naive Bayes conditional
        independence (i.i.d.) assumption as the sum of per-feature Gaussian
        log-likelihoods.


        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Input samples, where each row corresponds to a single data point.

        Returns
        -------
        ndarray of shape (n_samples, n_classes)
            Log posterior scores (not normalized).
        
        Notes
        -----
        - All computations are performed in log space for numerical stability.
        - This method assumes that the following attributes have already been
        estimated during fitting:
            - self.classes_
            - self.log_priors_
            - self.gaussian_means_
            - self.gaussian_vars_

        Algorithm
        ---------
        1. Initialize a matrix log_probs of shape (n_samples, n_classes)
        2. For each sample i:
            a. Extract the sample x_row = X[i]
            b. For each class c:
                i.   Compute log-likelihood log P(x | c) using _compute_log_likelihood
                ii.  Add the class log prior log P(c)
                iii. Store the result in log_probs[i, c]
        3. Return the log_probs matrix            

        Example
        -------
        >>> model = GaussianNaiveBayes()
        >>> model.classes_ = np.array(['A'])
        >>> model.log_priors_ = {'A': 0.0}
        >>> model.gaussian_means_ = {'A': {0: 0.0}}
        >>> model.gaussian_vars_  = {'A': {0: 1.0}}
        >>> model.predict_log_proba([[0]])
        array([[0.]])

        >>> model.classes_ = np.array(['A', 'B'])
        >>> model.log_priors_ = {'A': np.log(0.5), 'B': np.log(0.5)}
        >>> model.gaussian_means_ = {
        ...     'A': {0: 0.0},
        ...     'B': {0: 1.0}
        ... }
        >>> model.gaussian_vars_ = {
        ...     'A': {0: 1.0},
        ...     'B': {0: 1.0}
        ... }
        >>> model.predict_log_proba([[0.0], [1.0]])
        array([[-0.9189..., -1.4189...],
            [-1.4189..., -0.9189...]])
            
        """
        X = np.asarray(X, float)
        n_samples = X.shape[0]
        log_probs = np.zeros((n_samples, len(self.classes_)))

        for i in range(n_samples):
            x_row = X[i]
            for k, c in enumerate(self.classes_):
                log_l = self._compute_log_likelihood(x_row, c)
                log_probs[i, k] = self.log_priors_[c] + log_l

        return log_probs

    def predict_proba(self, X):
        """
        Compute normalized posterior probabilities P(Y = c | X) for each sample
        using the Gaussian Naive Bayes model.

        This method converts unnormalized log posterior scores obtained from
        `predict_log_proba` into valid probability distributions by applying
        the softmax function. To ensure numerical stability, the log-sum-exp
        trick is used by shifting log probabilities before exponentiation.

        Specifically, for each sample x:
            P(c | x) = exp(log P(c | x)) / sum_k exp(log P(k | x))
        where log P(c | x) = log P(c) + log P(x | c).

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)

        Returns
        -------
        ndarray of shape (n_samples, n_classes)
            Normalized probabilities. Each row sums to 1.

        Notes
        -----
        - This method relies on `predict_log_proba` to compute unnormalized
        log posterior scores.
        - For numerical stability, the maximum log posterior value is subtracted
        from each row before exponentiation:
            shifted = log_probs - max(log_probs)

        This does not change the resulting probabilities because softmax
        is invariant to constant shifts.

        Algorithm
        ---------
        1. Compute unnormalized log posterior scores using predict_log_proba(X)
        2. For each sample, subtract the maximum log score (numerical stability)
        3. Exponentiate the shifted log scores
        4. Normalize by dividing by the row-wise sum
        5. Return normalized probabilities


        Example
        -------
        >>> model = GaussianNaiveBayes()
        >>> model.classes_ = np.array(['A'])
        >>> model.log_priors_ = {'A': 0.0}
        >>> model.gaussian_means_ = {'A': {0: 0}}
        >>> model.gaussian_vars_  = {'A': {0: 1}}
        >>> model.predict_proba([[0]])
        array([[1.]])

        >>> model.classes_ = np.array(['A', 'B'])
        >>> model.log_priors_ = {'A': np.log(0.5), 'B': np.log(0.5)}
        >>> model.gaussian_means_ = {
        ...     'A': {0: 0.0},
        ...     'B': {0: 1.0}
        ... }
        >>> model.gaussian_vars_ = {
        ...     'A': {0: 1.0},
        ...     'B': {0: 1.0}
        ... }
        >>> model.predict_proba([[0.0], [1.0]])
        array([[0.6224..., 0.3775...],
            [0.3775..., 0.6224...]])
        """
        log_probs = self.predict_log_proba(X)
        max_log = np.max(log_probs, axis=1, keepdims=True)
        shifted = log_probs - max_log

        probs = np.exp(shifted)
        probs = probs / probs.sum(axis=1, keepdims=True)

        return probs

    def predict(self, X):
        """
        Predict the most likely class label for each sample.

        This method first computes normalized posterior probabilities using
        `predict_proba`, and then selects the class with the highest probability
        for each sample. Formally, the predicted class is:
            y_hat = argmax_c P(Y = c | x)

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)

        Returns
        -------
        ndarray of shape (n_samples,)
            Predicted class labels corresponding to the maximum posterior
            probability for each sample.

        Algorithm
        ---------
        1. Compute normalized posterior probabilities using predict_proba(X)
        2. Find the index of the maximum probability for each sample
        3. Map indices to class labels using self.classes_
        4. Return predicted class labels

        Example
        -------
        >>> X = np.array([[0], [2]])
        >>> model = GaussianNaiveBayes()
        >>> model.classes_ = np.array(['A'])
        >>> model.log_priors_ = {'A': 0.0}
        >>> model.gaussian_means_ = {'A': {0: 0}}
        >>> model.gaussian_vars_  = {'A': {0: 1}}
        >>> model.predict(X)
        array(['A', 'A'], dtype='<U1')
        """
        probs = self.predict_proba(X)
        class_indices = np.argmax(probs, axis=1)
        return self.classes_[class_indices]

## Result comparison with sklearn Gaussian Naive Bayes

To validate the correctness of our implementation, we compare it with sklearn.naive_bayes.GaussianNB on the Breast Cancer Wisconsin dataset. This dataset contains a binary target variable (malignant vs. benign) and 30 continuous numerical features, which makes it suitable for Gaussian Naive Bayes. The dataset is moderate in size (569 samples) and requires minimal preprocessing. This allows us to focus on verifying the correctness of the model implementation rather than handling extensive data cleaning or feature engineering, making it a good benchmark dataset for algorithm validation and comparison.

We apply the same preprocessing steps for both models: dropping the id column, mapping the binary target variable (M → 1, B → 0), removing missing values, and splitting the data into training and test sets using an 80/20 split with random_state=42 and stratify=y to preserve class proportions. Both models are trained on the identical training set, and we evaluate performance on the test set using accuracy and additionally compare predictions on a per-sample basis. 

In [59]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

df = pd.read_csv("data/data.csv")
df["diagnosis"] = df["diagnosis"].map({"M": 1, "B": 0})
df = df.dropna()

X = df.drop(columns=["id", "diagnosis"]).values.astype(float)
y = df["diagnosis"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

my_nb = GaussianNaiveBayes()
my_nb.fit(X_train, y_train)
y_pred_my = my_nb.predict(X_test)

sk_nb = GaussianNB(var_smoothing=1e-9)
sk_nb.fit(X_train, y_train)
y_pred_sk = sk_nb.predict(X_test)

print("My accuracy:", accuracy_score(y_test, y_pred_my))
print("sklearn accuracy:", accuracy_score(y_test, y_pred_sk))

diff_mask = (y_pred_my != y_pred_sk)
num_diff = diff_mask.sum()
print("Number of difference:", num_diff)

diff_indices = np.where(diff_mask)[0]
print("Different number index:", diff_indices)

print("True y:", y_test[diff_indices])
print("My y:      ", y_pred_my[diff_indices])
print("sklearn y: ", y_pred_sk[diff_indices])

My accuracy: 0.9385964912280702
sklearn accuracy: 0.9385964912280702
Number of difference: 0
Different number index: []
True y: []
My y:       []
sklearn y:  []


In [60]:
probs_my = my_nb.predict_proba(X_test)
probs_sk = sk_nb.predict_proba(X_test)

prob_diff = np.abs(probs_my - probs_sk)
max_diff = prob_diff.max()

assert np.allclose(probs_my, probs_sk, atol=1e-10), "Predicted probabilities differ from sklearn beyond tolerance."

print("Probability comparison PASSED: predicted probabilities match sklearn within numerical tolerance.")


Probability comparison PASSED: predicted probabilities match sklearn within numerical tolerance.


The results show that our implementation and sklearn’s implementation achieve exactly the same accuracy (0.9386), and there are zero differences in predicted class labels across all test samples. In addition, the posterior class probabilities produced by both models also match for every test instance. Together, these results demonstrate that our Gaussian Naive Bayes model not only matches sklearn in aggregate performance, but also exactly reproduces both its predictions and probabilistic outputs on this dataset.

## Check Model

In this section, we validate the correctness and robustness of our Gaussian Naive Bayes implementation through a set of unit tests to ensure that every function works correctly, and also under edge cases. 


# Check 1: `_gaussian_log_pdf`

**Test 1.1 : Gaussian log pdf at the mean**

checks that our implementation of `_gaussian_log_pdf` matches the closed-form analytical value at the mean.  

For $N(0, 1)$, log pdf at $x=0$ is $-0.5 * \log(2*pi)$


In [61]:
import pytest
m = GaussianNaiveBayes()

val = m._gaussian_log_pdf(0.0, mean=0.0, var=1.0)

expected = -0.5 * np.log(2.0 * np.pi * 1.0)

print("Computed:", val)
print("Expected:", expected)
assert val == pytest.approx(expected, rel=1e-6)
print("Test 1.1 PASSED: gaussian_log_pdf center value correct.")



Computed: -0.9189385332046727
Expected: -0.9189385332046727
Test 1.1 PASSED: gaussian_log_pdf center value correct.


**Test 1.2: Monotonic decay away from the mean**

checks that our implementation of `_gaussian_log_pdf` gives correct Gaussian log density result that will monotonic decay as we move away from the mean.

For a fixed normal distribution $N(0, 1)$, the log pdf should be highest at the mean and decrease as $|x - \mu|$ increases.


In [62]:
center = m._gaussian_log_pdf(0.0, mean=0.0, var=1.0)
off_1  = m._gaussian_log_pdf(1.0, mean=0.0, var=1.0)
off_2  = m._gaussian_log_pdf(2.0, mean=0.0, var=1.0)
print(center, off_1, off_2)

assert center > off_1 > off_2
print("Test 1.2 PASSED: gaussian_log_pdf monotonicity correct.")


-0.9189385332046727 -1.4189385332046727 -2.9189385332046727
Test 1.2 PASSED: gaussian_log_pdf monotonicity correct.


**Test 1.3: Effect of the variance on the log pdf**

checks that changing the variance parameter in the Gaussian distribution changes the output of `_gaussian_log_pdf`.

In [63]:
n1 = m._gaussian_log_pdf(1.0, mean=0.0, var=0.5)
n2 = m._gaussian_log_pdf(1.0, mean=0.0, var=5.0)

assert not np.isclose(n1, n2)
print("Test 1.3 PASSED: variance affects gaussian_log_pdf output.")


Test 1.3 PASSED: variance affects gaussian_log_pdf output.


# Check 2: `_compute_log_likelihood`

**Test 2.1: calculation of log likelihood value**

checks if `_compute_log_likelihood` correctly sums the per-feature Gaussian log pdfs by manually calculating total log-likelihood from a 2-dimensional toy dataset example.

In [64]:
m.classes_ = np.array([0])
m.gaussian_means_[0] = {0: 0.0, 1: 1.0}
m.gaussian_vars_[0]  = {0: 1.0, 1: 4.0}

x_row = np.array([0.0, 3.0])
log1 = m._gaussian_log_pdf(x_row[0], 0.0, 1.0)
log2 = m._gaussian_log_pdf(x_row[1], 1.0, 4.0)
expected = log1 + log2

assert m._compute_log_likelihood(x_row, 0) == pytest.approx(expected, rel=1e-6)
print("Test 2.1 PASSED: compute_log_likelihood matches manual calculation.")


Test 2.1 PASSED: compute_log_likelihood matches manual calculation.


**Test 2.2： distance and log-likelihood**

checks that samples farther from the class mean produce lower log-likelihood values. 

In [65]:
x1 = np.array([0.0, 1.0])
x2 = np.array([5.0, 10.0])

ll1 = m._compute_log_likelihood(x1, 0)
ll2 = m._compute_log_likelihood(x2, 0)

assert ll1 > ll2
print("Test 2.2 PASSED: log-likelihood decreases for far-away samples.")


Test 2.2 PASSED: log-likelihood decreases for far-away samples.


# Check 3: `fit`

**Test 3.1: Correct computation of class priors and class structure**

checks that `fit()` correctly detects class labels and calculates the corresponding log-priors based on class frequencies. 


In [66]:
X_toy = np.array([[1,2],[1.2,1.9],[3.2,4.8],[3,5.1],[2.5,2.9]])
y_toy = np.array([0,0,1,1,1])

m.fit(X_toy, y_toy)

# classes
assert isinstance(m.classes_, np.ndarray)
assert m.classes_.ndim == 1
assert set(m.classes_) == {0,1}

assert set(m.log_priors_.keys()) == set(m.classes_)


N = len(y_toy)
for c in m.classes_:
    expected = np.log(np.sum(y_toy == c) / N)
    assert np.isclose(m.log_priors_[c], expected)

print("Test 3.1 PASSED: fit() correctly computes priors and classes.")


Test 3.1 PASSED: fit() correctly computes priors and classes.


**Test 3.2: Positive variances after smoothing**

checks that all per-feature variances are strictly positive after smoothing. 


In [67]:

for c in m.classes_:
    for j in range(X_toy.shape[1]):
        assert m.gaussian_vars_[c][j] > 0

print("Test 3.2 PASSED: fit() produces positive variances.")


Test 3.2 PASSED: fit() produces positive variances.


**Test 3.3: Correct structure of stored means and variances**

checks that each class stores a full set of per-feature means and variances with the correct dictionary structure.

In [68]:

n_features = X_toy.shape[1]

for c in m.classes_:
    assert len(m.gaussian_means_[c]) == n_features
    assert len(m.gaussian_vars_[c]) == n_features

print("Test 3.3 PASSED: fit() structure for means/vars correct.")


Test 3.3 PASSED: fit() structure for means/vars correct.


**Test 3.4: NaN in input should raise ValueError**

when there's NaN value in fit() input, raise ValueError. This ValueError is included in the model since GNB could not process NaN values.


In [69]:
try:
    X_nan = np.array([[1.0, 2.0],
                      [np.nan, 3.0]])
    y_nan = np.array([0, 1])

    m.fit(X_nan, y_nan)
    raise AssertionError("Test 3.4 FAILED: fit did not raise ValueError on NaN input.")
except ValueError:
    print("Test 3.4 PASSED: fit raises ValueError when X contains NaN.")


Test 3.4 PASSED: fit raises ValueError when X contains NaN.


# Check 4: `predict_log_proba`

**Test 4.1: Output shape and dtype of `predict_log_proba`**

checks that `predict_log_proba` method returns the output shape (n_sample, n_classes) and dtype.  



In [70]:

X_test = np.array([[1.1,2.0],[3.1,4.9]])

logp = m.predict_log_proba(X_test)
assert isinstance(logp, np.ndarray)
assert logp.shape == (2,2)
assert logp.dtype == float

print("Test 4.1 PASSED: predict_log_proba shape + dtype correct.")


Test 4.1 PASSED: predict_log_proba shape + dtype correct.


**Test 4.2: `predict_log_proba` scores**

checks that the more distant the samples, the lower the log-posterior scores.


In [71]:
lp1 = m.predict_log_proba([[1.1,2.0]])[0]
lp2 = m.predict_log_proba([[100,100]])[0]

assert lp1.max() > lp2.max()
print("Test 4.2 PASSED: predict_log_proba scores changes correspond to the distance.")


Test 4.2 PASSED: predict_log_proba scores changes correspond to the distance.


# Check 5: `predict_proba`

**Test 5.1: Probability normalization**

check that each probability row from `predict_proba` sums to 1.


In [72]:

probs = m.predict_proba(X_test)
assert np.allclose(probs.sum(axis=1), 1.0)

print("Test 5.1 PASSED: predict_proba rows sum to 1.")


Test 5.1 PASSED: predict_proba rows sum to 1.


**Test 5.2: Non negative probability**

checks all values returned by `predict_proba` are non-negative.


In [73]:
assert np.all(probs >= 0)
print("Test 5.2 PASSED: predict_proba produces non-negative probability.")


Test 5.2 PASSED: predict_proba produces non-negative probability.


**Test 5.3: Consistency between log-probability and probability**

checks that the class with the highest log-probability also has the highest probability after normalization.

In [74]:
# class with max log-prob = class with max prob
logp = m.predict_log_proba(X_test)
probs = m.predict_proba(X_test)

assert np.argmax(logp[0]) == np.argmax(probs[0])
print("Test 5.3 PASSED: predict_proba aligns with predict_log_proba.")


Test 5.3 PASSED: predict_proba aligns with predict_log_proba.


# Check 6: `predict`

**Test 6.1: Output shape and valid class labels**

checks that `predict` returns a 1D array of valid class labels.


In [75]:
preds = m.predict(X_test)

assert preds.shape == (X_test.shape[0],)
assert set(preds).issubset(set(y_toy))

print("Test 6.1 PASSED: predict shape and value set correct.")

Test 6.1 PASSED: predict shape and value set correct.


**Test 6.2: Consistency with `predict_proba`**

checks that `predict` always chooses the class with the highest predicted probability.


In [76]:
probs = m.predict_proba(X_test)
assert np.array_equal(preds, np.argmax(probs, axis=1))

print("Test 6.2 PASSED: predict matches argmax of probability.")


Test 6.2 PASSED: predict matches argmax of probability.


# Check 7: Edge cases

**Test 7.1: zero-variance feature**

checks the case where a feature has variance 0 within each class. Confirms variance smoothing prevents divide-by-zero and that predict_log_proba()/predict_proba() remain finite and normalized. 

In [77]:

X_zero_var = np.array([
    [1.0, 2.0],
    [1.0, 3.0],
    [1.0, 4.0],
    [1.0, 5.0],
])
y_zero_var = np.array([0, 0, 1, 1])

m.fit(X_zero_var, y_zero_var)

logp_zero = m.predict_log_proba(X_zero_var)
probs_zero = m.predict_proba(X_zero_var)

assert np.isfinite(logp_zero).all(), "Test 7.1 FAILED: predict_log_proba produced non-finite values with zero-variance feature."
assert np.isfinite(probs_zero).all(), "Test 7.1 FAILED: predict_proba produced non-finite values with zero-variance feature."
assert np.allclose(probs_zero.sum(axis=1), 1.0, atol=1e-6), "Test 7.1 FAILED: probabilities do not sum to ~1."

print("Test 7.1 PASSED: zero-variance features are handled correctly via variance smoothing.")



Test 7.1 PASSED: zero-variance features are handled correctly via variance smoothing.


**Test 7.2: Single-class training**

checks that training on data with only one class does not crash and produces reasonable outputs. Ensures classes_ has length 1, predict_proba() returns probabilities ≈ 1, and predict() always returns the only class.

In [78]:
X_one_class = np.array([
    [1.0, 2.0],
    [1.5, 2.5],
    [2.0, 3.0],
])
y_one_class = np.array([0, 0, 0])

m.fit(X_one_class, y_one_class)

assert m.classes_.shape == (1,), "Test 7.2 FAILED: classes_ should contain exactly one class."
assert m.classes_[0] == 0, "Test 7.2 FAILED: the single class in classes_ should be 0."

probs_one_class = m.predict_proba(X_one_class)
preds_one_class = m.predict(X_one_class)

assert probs_one_class.shape == (X_one_class.shape[0], 1), "Test 7.2 FAILED: predict_proba shape incorrect for single-class training."
assert np.allclose(probs_one_class, 1.0, atol=1e-6), "Test 7.2 FAILED: probabilities are not all ~1 for single-class training."
assert np.array_equal(preds_one_class, np.zeros_like(y_one_class)), "Test 7.2 FAILED: predict did not return the only class."

print("Test 7.2 PASSED: model behaves correctly when trained on a single class.")

Test 7.2 PASSED: model behaves correctly when trained on a single class.


**Test 7.3: Single test sample**

verifies shape handling and probability normalization when predicting on a single input sample. Ensures predict_proba() returns shape (1, C), sums to 1, and predict() returns a valid class label.

In [79]:
X_train_small = np.array([
    [2.0, 3.0],
    [3.0, 4.0],
    [4.0, 5.0],
])
y_train_small = np.array([0, 1, 1])
X_test_single = np.array([[3.5, 4.5]])

m.fit(X_train_small, y_train_small)

probs_test_single = m.predict_proba(X_test_single)
pred_test_single = m.predict(X_test_single)

assert probs_test_single.shape == (1, 2), "Test 7.3 FAILED: predict_proba shape incorrect for a single test sample."
assert np.allclose(probs_test_single.sum(axis=1), 1.0, atol=1e-6), "Test 7.3 FAILED: probs do not sum to ~1."
assert pred_test_single.shape == (1,), "Test 7.3 FAILED: predict shape incorrect for a single test sample."
assert pred_test_single[0] in m.classes_, "Test 7.3 FAILED: predicted label not in known classes."

print("Test 7.3 PASSED: predict_proba/predict handle a single test sample correctly.")


Test 7.3 PASSED: predict_proba/predict handle a single test sample correctly.


**Test 7.4: Extreme magnitudes**

tests numerical stability under very large feature values. Confirms log-sum-exp and log-space computations avoid underflow/overflow and keep outputs finite and normalized.

Large values stress (x-mean)^2 / var. We only require finite outputs and normalized probabilities.

In [80]:

X_extreme = np.array([
    [ 1e9,  -1e9],
    [ 1e9+1, -1e9-1],
    [-1e9,   1e9],
    [-1e9-1, 1e9+1],
])
y_extreme = np.array([0, 0, 1, 1])

m.fit(X_extreme, y_extreme)

logp_ext = m.predict_log_proba(X_extreme)
probs_ext = m.predict_proba(X_extreme)

assert np.isfinite(logp_ext).all(), "Test 7.4 FAILED: log-probabilities contain NaN/inf under extreme magnitudes."
assert np.isfinite(probs_ext).all(), "Test 7.4 FAILED: probabilities contain NaN/inf under extreme magnitudes."
assert np.allclose(probs_ext.sum(axis=1), 1.0, atol=1e-6), "Test 7.4 FAILED: probs do not sum to ~1 under extreme magnitudes."

print("Test 7.4 PASSED: extreme magnitudes remain numerically stable (log-space + log-sum-exp).")


Test 7.4 PASSED: extreme magnitudes remain numerically stable (log-space + log-sum-exp).


Overall, these tests demonstrates that each component of our Gaussian Naive Bayes implementation works correctly in isolation, handles edge cases correctly, and exactly reproduces both the predictions and probabilistic outputs of sklearn’s implementation on a real-world dataset. 